<a href="https://colab.research.google.com/github/nish1606/Image-segmentation-Comp-vision/blob/main/image_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from ipywidgets import interact, IntSlider, Dropdown, Text
from IPython.display import display

# Helper functions for image processing
def to_grayscale(image):
    """Converts a BGR image to grayscale."""
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def basic_global_thresholding(image, threshold_value):
    """Applies basic global thresholding."""
    gray_image = to_grayscale(image)
    _, segmented = cv2.threshold(gray_image, threshold_value, 255, cv2.THRESH_BINARY)
    return segmented

def sobel_edge_detection(image):
    """Applies Sobel edge detection."""
    gray_image = to_grayscale(image)
    sobelx = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = cv2.magnitude(sobelx, sobely)
    # Normalize and convert back to 8-bit image
    gradient_magnitude = np.uint8(255 * gradient_magnitude / np.max(gradient_magnitude))
    return gradient_magnitude

def region_growing(image, seed_point, similarity_threshold):
    """Performs region growing segmentation."""
    gray_image = to_grayscale(image)
    height, width = gray_image.shape
    segmented = np.zeros_like(gray_image)
    visited = np.zeros_like(gray_image, dtype=bool)

    if not (0 <= seed_point[0] < width and 0 <= seed_point[1] < height):
        return segmented

    seed_value = gray_image[seed_point[1], seed_point[0]]
    stack = [seed_point]

    while stack:
        x, y = stack.pop()

        if not visited[y, x]:
            visited[y, x] = True
            segmented[y, x] = 255

            # Check neighbors
            for dy, dx in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nx, ny = x + dx, y + dy

                if 0 <= nx < width and 0 <= ny < height and not visited[ny, nx]:
                    neighbor_value = gray_image[ny, nx]
                    if abs(int(neighbor_value) - int(seed_value)) < similarity_threshold:
                        stack.append((nx, ny))
    return segmented

def region_splitting_merging(image, threshold=20):
    """
    Illustrative region splitting and merging using a simplified quadtree.
    Splits if variance is above threshold, merges by filling with average color.
    """
    gray_image = to_grayscale(image)
    height, width = gray_image.shape
    segmented = np.zeros_like(gray_image)

    def split_and_merge(x, y, w, h):
        region = gray_image[y:y+h, x:x+w]
        if region.size > 0:
            if region.std() > threshold and (w > 1 and h > 1):
                # Split
                split_and_merge(x, y, w // 2, h // 2)
                split_and_merge(x + w // 2, y, w - w // 2, h // 2)
                split_and_merge(x, y + h // 2, w // 2, h - h // 2)
                split_and_merge(x + w // 2, y + h // 2, w - w // 2, h - h // 2)
            else:
                # Merge
                segmented[y:y+h, x:x+w] = np.mean(region)

    split_and_merge(0, 0, width, height)
    return segmented

# Main interactive function for the Colab notebook
def interactive_segmentation(image_path):
    # Load the image
    original_image = cv2.imread(image_path)
    if original_image is None:
        print("Error: Could not read image. Please check the file path.")
        return

    original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    # Widget setup
    method_selector = Dropdown(
        options=['None', 'Thresholding', 'Edge Detection', 'Region Growing', 'Region Splitting & Merging'],
        value='None',
        description='Method:',
    )

    threshold_slider = IntSlider(min=0, max=255, step=1, value=128, description='Threshold:')

    similarity_threshold_slider = IntSlider(min=1, max=100, step=1, value=25, description='Similarity:')

    seed_point_text = Text(value='100,100', description='Seed Point (x,y):')

    # The main callback function for the interactive widget
    def apply_method(method, threshold, similarity, seed_text):
        try:
            x, y = map(int, seed_text.split(','))
            seed_point = (x, y)
        except ValueError:
            print("Invalid seed point format. Please use 'x,y'.")
            return

        processed_image = original_image_rgb.copy()

        if method == 'Thresholding':
            processed_image = basic_global_thresholding(original_image, threshold)
        elif method == 'Edge Detection':
            processed_image = sobel_edge_detection(original_image)
        elif method == 'Region Growing':
            processed_image = region_growing(original_image, seed_point, similarity)
        elif method == 'Region Splitting & Merging':
            processed_image = region_splitting_merging(original_image, threshold=threshold)

        # Display images
        plt.figure(figsize=(12, 6))

        plt.subplot(1, 2, 1)
        plt.title("Original Image")
        plt.imshow(original_image_rgb)
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title(f"Processed Image ({method})")
        plt.imshow(processed_image, cmap='gray' if method != 'None' else None)
        plt.axis('off')

        plt.show()

    # Create the interactive interface
    interact(apply_method, method=method_selector, threshold=threshold_slider,
             similarity=similarity_threshold_slider, seed_text=seed_point_text)

# --- INSTRUCTIONS FOR USE ---
# 1. Upload an image to your Colab session's file system (e.g., 'sample.jpg').
# 2. Call the function with your image's filename.
#    Example: interactive_segmentation('sample.jpg')

# --- START THE PROJECT ---
# Replace 'sample.jpg' with the name of your uploaded image file.
# interactive_segmentation('sample.jpg') # Uncomment this line to run
interactive_segmentation('/content/image1.webp')

interactive(children=(Dropdown(description='Method:', options=('None', 'Thresholding', 'Edge Detection', 'Regi…